In [1]:
#!pip install pyreadstat
import pyreadstat
import pandas as pd

df_path = "anes_timeseries_2020_stata_20210324.dta"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, bernoulli, beta, norm
from scipy.special import expit as logistic_sigmoid
import statsmodels.api as sm
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [13]:
def find_unique(column):
    unique_values = df[column].unique()
    print(f"Unique values in df[{column}]:")
    print(unique_values)

In [16]:
# Read file
df, meta = pyreadstat.read_dta(df_path)
print(len(df['V200001'].unique())) # number of unique people who responded, should be equal to length of df before filtering

# Keep only observations from respondents who say they intend to vote
print("Shape before filtering:", df.shape)
df = df[df['V201032'].isin([1.])]
print("Shape after filtering:", df.shape)

# Define the outcome of interest as 1 if the voter will vote for the Republicans, 0 otherwise
df = df[(df['V201033'] != -9) & (df['V201033'] != -8)]
trump_votes_count = df['V201033'].value_counts().get(2.0, 0)
print("Number of people who prefer Trump:", trump_votes_count)

# Create binary outcome variable Y
Y = df['V201033'].apply(lambda x: 1 if x == 2.0 else 0)

find_unique('V201549x')

# Filter out missing values from X_df based on indices of Y
"""
X_df = pd.DataFrame({
    'Age': pd.to_numeric(df['V201507x'], errors='coerce'),
    'College_Degree_or_Higher': df['V201510'].apply(lambda x: 1 if x in ["6. Bachelor's degree (e.g. BA, AB, BS)", "7. Master's degree (e.g. MA, MS, MEng, MEd, MSW, MBA)", "4. Associate degree in college - occupational/vocational", "5. Associate degree in college - academic"] else 0),
    'Trust_in_Media': df['V201377'].apply(lambda x: 1 if (x == 5 or x == 4 or x == 3) else 0),
    'Sex_Female': df['V201600'].apply(lambda x: 1 if x == '2. Female' else 0),
    'Ethnicity_NonWhite': df['V201549x'].apply(lambda x: 1 if x == '1. White, non-Hispanic' else 0),
    'Sexual_Orientation_Lgbtq': df['V201601'].apply(lambda x: 1 if (x == 2 or x == 3 or x == 4) else 0),
})
"""
X_df = pd.DataFrame({
    'Age': pd.to_numeric(df['V201507x'], errors='coerce'),
    'College_Degree_or_Higher': df['V201510'].apply(lambda x: 1 if x in [6, 7, 4, 5] else 0),
    'Trust_in_Media': df['V201377'].apply(lambda x: 1 if (x == 5 or x == 4 or x == 3) else 0),
    'Sex_Female': df['V201600'].apply(lambda x: 1 if x == 2 else 0),
    'Ethnicity_NonWhite': df['V201549x'].apply(lambda x: 1 if x == 1 else 0),
    'Sexual_Orientation_Lgbtq': df['V201601'].apply(lambda x: 1 if (x == 2 or x == 3 or x == 4) else 0),
})

# Ensure consistent indices and drop missing values
X_df = X_df.dropna()
Y = Y.dropna()
Y = Y.loc[X_df.index]
print(f'size of Y: {len(Y)}')
print(f'size of X_df: {len(X_df)}')

# Create design matrix
X_with_intercept = sm.add_constant(X_df)
print(f'size of X_with_intercept: {len(X_with_intercept)}')

print(X_with_intercept.head())

8280
Shape before filtering: (8280, 1381)
Shape after filtering: (7272, 1381)
Number of people who prefer Trump: 3016
Unique values in df[V201549x]:
[ 3.  4.  1.  5.  2. -9.  6. -8.]
size of Y: 7138
size of X_df: 7138
size of X_with_intercept: 7138
   const   Age  College_Degree_or_Higher  Trust_in_Media  Sex_Female  \
0    1.0  46.0                         1               0           0   
1    1.0  37.0                         0               0           1   
2    1.0  40.0                         0               1           1   
3    1.0  41.0                         1               1           0   
4    1.0  72.0                         0               0           0   

   Ethnicity_NonWhite  Sexual_Orientation_Lgbtq  
0                   0                         0  
1                   0                         0  
2                   1                         0  
3                   0                         0  
4                   0                         0  


In [17]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate VIF for each predictor variable
vif_data = X_with_intercept.assign(const=1)
vif_series = pd.Series([variance_inflation_factor(vif_data.values, i) 
                        for i in range(vif_data.shape[1])], 
                       index=vif_data.columns)

print("Variance Inflation Factors:")
print(vif_series)


Variance Inflation Factors:
const                       12.325642
Age                          1.060887
College_Degree_or_Higher     1.010899
Trust_in_Media               1.039232
Sex_Female                   1.003215
Ethnicity_NonWhite           1.061881
Sexual_Orientation_Lgbtq     1.020873
dtype: float64


In [18]:
# Fit the model 
model = sm.Logit(Y, X_with_intercept).fit()

Optimization terminated successfully.
         Current function value: 0.474917
         Iterations 6


In [19]:
# Get summary results
summary = model.summary()
print(summary)

                           Logit Regression Results                           
Dep. Variable:                V201033   No. Observations:                 7138
Model:                          Logit   Df Residuals:                     7131
Method:                           MLE   Df Model:                            6
Date:                Tue, 05 Mar 2024   Pseudo R-squ.:                  0.3027
Time:                        10:16:49   Log-Likelihood:                -3390.0
converged:                       True   LL-Null:                       -4861.7
Covariance Type:            nonrobust   LLR p-value:                     0.000
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       -0.0691      0.104     -0.664      0.506      -0.273       0.135
Age                          0.0120      0.002      7.720      0.000       0.009       0.

In [35]:
# Extract the coefficients (betas) and their covariance matrix from the logistic regression fit
beta_mean = model.params
beta_cov = model.cov_params()
display(beta_mean)
display(beta_cov)

const                      -0.069116
Age                         0.012011
College_Degree_or_Higher   -0.426257
Trust_in_Media             -2.752106
Sex_Female                 -0.251244
Ethnicity_NonWhite          1.000446
Sexual_Orientation_Lgbtq   -1.476073
dtype: float64

,const,Age,College_Degree_or_Higher,Trust_in_Media,Sex_Female,Ethnicity_NonWhite,Sexual_Orientation_Lgbtq
const,0.010825,-1.035116e-04,-0.002098,-0.000420,-1.888064e-03,-0.003045,-0.002188
Age,-0.000104,2.420685e-06,0.000006,-0.000023,-2.627104e-07,-0.000016,0.000023
College_Degree_or_Higher,-0.002098,5.640243e-06,0.003752,0.000277,-6.388926e-05,-0.000405,0.000428
Trust_in_Media,-0.000420,-2.250010e-05,0.000277,0.004268,8.438820e-05,-0.000263,0.000534
Sex_Female,-0.001888,-2.627104e-07,-0.000064,0.000084,3.679932e-03,-0.000080,-0.000207
Ethnicity_NonWhite,-0.003045,-1.635838e-05,-0.000405,-0.000263,-8.015244e-05,0.005446,-0.000284
Sexual_Orientation_Lgbtq,-0.002188,2.265766e-05,0.000428,0.000534,-2.072090e-04,-0.000284,0.023682


In [21]:
# Number of simulations
n_simulations = 1000

# Simulate beta coefficients:

# on the log-odds scale
simulated_base_log_odds = multivariate_normal.rvs(mean=beta_mean, cov=beta_cov, size=n_simulations)

# on the odds scale
simulated_base_odds = np.exp(simulated_base_log_odds)

# on the probability scale
simulated_base_prob = logistic_sigmoid(simulated_base_log_odds)

# simulations array
simulated_base = np.stack((simulated_base_log_odds, simulated_base_odds, simulated_base_prob), axis=1)

In [32]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate statistics for each coefficient
medians = np.median(simulated_base, axis=0)
lower_quantiles = np.percentile(simulated_base, 5, axis=0)
upper_quantiles = np.percentile(simulated_base, 95, axis=0)
references = [0, 1, 0.5]  # Reference values for log-odds, odds, probability
prob_positive = [np.mean(simulated_base[:, i] > references[i]) for i in range(3)]

# Plot names 
col_names = ['Log-Odds', 'Odds', 'Probability']

# Plot histograms of each metric
fig, axs = plt.subplots(2, 2, figsize=(15, 10))  # Adjust for the number of metrics
axs = axs.flatten()

for i in range(len(col_names)):
    ax = axs[i]
    ax.hist(simulated_base[:, i], bins=30, density=True, alpha=0.7)
    
    # Add vertical lines for reference, median, and 90% interval
    ax.axvline(references[i], color='red', linestyle='-', label=f'Reference: {references[i]}')
    ax.axvline(float(medians[i][0]), color='orange', linestyle='-', label=f'Median: {float(medians[i][0]:.2f)}')
    ax.axvline(lower_quantiles[i], color='orange', linestyle='--', label=f'5th Percentile: {lower_quantiles[i]:.2f}')
    ax.axvline(upper_quantiles[i], color='orange', linestyle='--', label=f'95th Percentile: {upper_quantiles[i]:.2f}')
    ax.text(0.05, 0.95, f'Prob > Ref: {prob_positive[i]*100:.1f}%', transform=ax.transAxes, verticalalignment='top')
    
    ax.set_title(f'Average {col_names[i]} of Voting Republican')
    ax.set_ylabel('Density')
    ax.legend(loc='upper right')

# Hide empty subplot for uneven number of metrics
if len(col_names) % 2 != 0:
    axs[-1].axis('off')
    
plt.tight_layout()
plt.show()


SyntaxError: f-string: invalid syntax (2155074747.py, line 24)

In [ ]:
aic = model.aic
print("AIC:", aic)

In [33]:
def logistic_regression(X, y, test_size=0.2, random_state=None):
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Initialize logistic regression model
    log_reg_model = LogisticRegression()

    # Fit the model on the training data
    log_reg_model.fit(X_train, y_train)

    # Predictions on the test data
    y_pred = log_reg_model.predict(X_test)

    # Evaluate the model
    classification_rep = classification_report(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    return classification_rep, conf_matrix, accuracy

classification_rep, conf_matrix, accuracy = logistic_regression(X_with_intercept, Y, test_size=0.3, random_state=420)
print("Classification Report:")
print(classification_rep)
print("\nConfusion Matrix:")
print(conf_matrix)
print(f'Accuracy:  {accuracy:.4f}')

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.81      0.82      1206
           1       0.76      0.79      0.78       936

    accuracy                           0.80      2142
   macro avg       0.80      0.80      0.80      2142
weighted avg       0.80      0.80      0.80      2142


Confusion Matrix:
[[974 232]
 [196 740]]
Accuracy:  0.8002
